In [1]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [2]:
from praw.models import user
import praw
import pandas as pd
import prawcore.exceptions

In [3]:
# Initialize PRAW with your API credentials
reddit = praw.Reddit(
client_id="nFKOCvQQEIoW2hFeVG6kfA",
client_secret="5BBB4fr-HMPtO8f4jZhle74-fYcDkQ",
user_agent="Icy_Process3191",
)

In [4]:
# not necessary
'''
def get_users_with_post_count(subreddit_name, n, reddit):

    user_post_counts = {}

    subreddit = reddit.subreddit(subreddit_name)
    for submission in subreddit.new(limit=None):
        author = submission.author
        if author:
            author_name = str(author)
            if author_name in user_post_counts:
                user_post_counts[author_name] += 1
            else:
                user_post_counts[author_name] = 1

    users_more_than_n = [user for user, post_count in user_post_counts.items() if post_count > n]

    return users_more_than_n
'''


'\ndef get_users_with_post_count(subreddit_name, n, reddit):\n\n    user_post_counts = {}\n\n    subreddit = reddit.subreddit(subreddit_name)\n    for submission in subreddit.new(limit=None):\n        author = submission.author\n        if author:\n            author_name = str(author)\n            if author_name in user_post_counts:\n                user_post_counts[author_name] += 1\n            else:\n                user_post_counts[author_name] = 1\n\n    users_more_than_n = [user for user, post_count in user_post_counts.items() if post_count > n]\n\n    return users_more_than_n\n'

In [5]:
# not necessary
'''
def get_user_subreddit_metrics(username, reddit):

    subreddit_metrics = {}

    try:
        redditor = reddit.redditor(username)
    except praw.exceptions.APIException as e:
        print(f"Error: {e}")
        return None

    # Iterate through the user's submissions (posts)
    for submission in redditor.submissions.new(limit=None):
        subreddit_name = submission.subreddit.display_name

        if subreddit_name not in subreddit_metrics:
            subreddit_metrics[subreddit_name] = {
                'Number of Posts': 0,
                'Total Karma': 0,
                'Total Comments': 0
            }


        subreddit_metrics[subreddit_name]['Number of Posts'] += 1
        subreddit_metrics[subreddit_name]['Total Karma'] += submission.score
        subreddit_metrics[subreddit_name]['Total Comments'] += submission.num_comments


    df = pd.DataFrame.from_dict(subreddit_metrics, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Subreddit'}, inplace=True)

    return df
'''

'\ndef get_user_subreddit_metrics(username, reddit):\n\n    subreddit_metrics = {}\n\n    try:\n        redditor = reddit.redditor(username)\n    except praw.exceptions.APIException as e:\n        print(f"Error: {e}")\n        return None\n\n    # Iterate through the user\'s submissions (posts)\n    for submission in redditor.submissions.new(limit=None):\n        subreddit_name = submission.subreddit.display_name\n\n        if subreddit_name not in subreddit_metrics:\n            subreddit_metrics[subreddit_name] = {\n                \'Number of Posts\': 0,\n                \'Total Karma\': 0,\n                \'Total Comments\': 0\n            }\n\n\n        subreddit_metrics[subreddit_name][\'Number of Posts\'] += 1\n        subreddit_metrics[subreddit_name][\'Total Karma\'] += submission.score\n        subreddit_metrics[subreddit_name][\'Total Comments\'] += submission.num_comments\n\n\n    df = pd.DataFrame.from_dict(subreddit_metrics, orient=\'index\')\n    df.reset_index(inplace=

In [6]:
# not necessary
#get_user_subreddit_metrics("Quiet_Effective7234",reddit)

In [7]:
def get_user_metrics(usernames):

    user_metrics = {}

    for username in set(usernames):

        user = reddit.redditor(username)
        user_metrics[username] = {
            'Username': username,
            'Comment Karma': user.comment_karma,
            'Post Karma': user.link_karma,
            'Post Count': sum(1 for _ in user.submissions.new(limit=None)),
            'Comment Count': sum(1 for _ in user.comments.new(limit=None))
        }

    return user_metrics

In [14]:
def get_posts(sub_name, n, top_comments_count):
    subreddit = reddit.subreddit(sub_name)
    top_posts = subreddit.top(time_filter='week', limit=n)

    post_data = []
    comment_data = []
    author_usernames = []

    for post in top_posts:
        # by hiding the below we can get comments on non-self posts
        #if not post.is_self:
        #    continue

        author_username = post.author.name if post.author else None
        author_usernames.append(author_username)

        post_data.append({
            'Author': author_username,
            'Subreddit': post.subreddit.display_name,
            'post_id': post.id,
            'Score': post.score,
            'Upvotes': post.ups,
            'Downvotes': post.downs,
            'Number of Comments': post.num_comments,
            'Text': post.selftext,
            'Created On': post.created_utc,
        })

        submission = reddit.submission(id=post.id)
        submission.comment_sort = 'best'
        submission.comments.replace_more(limit=0)

        for comment in submission.comments.list()[:top_comments_count]:
            if hasattr(comment, "author") and comment.author:
                comment_author = comment.author.name
                author_usernames.append(comment_author)
                comment_data.append({
                    'post_id': post.id,
                    'comment_text': comment.body,
                    'comment_score': comment.score,
                    'comment_ups': comment.ups,
                    'comment_downs': comment.downs,
                    'comment_author': comment_author
                })

    '''
    not necessary:
    user_metrics = get_user_metrics(author_usernames)

    for comment in comment_data:
        username = comment['comment_author']
        comment.update(user_metrics.get(username, {}))
    '''

    # get dfs into one df
    post_df = pd.DataFrame(post_data)
    comment_df = pd.DataFrame(comment_data)
    combined_df = pd.merge(post_df, comment_df, on='post_id')
    
    return combined_df
    

In [15]:
combined_df = get_posts(sub_name='funny', n=3, top_comments_count=3)

In [16]:
print(combined_df)

              Author Subreddit  post_id  Score  Upvotes  Downvotes   
0  DevelopmentNo9622     funny  185hd3g  44666    44666          0  \
1  DevelopmentNo9622     funny  185hd3g  44666    44666          0   
2  DevelopmentNo9622     funny  185hd3g  44666    44666          0   
3          lolikroli     funny  183kfng  42274    42274          0   
4          lolikroli     funny  183kfng  42274    42274          0   
5          lolikroli     funny  183kfng  42274    42274          0   
6            flavioj     funny  184qb6x  38424    38424          0   
7            flavioj     funny  184qb6x  38424    38424          0   
8            flavioj     funny  184qb6x  38424    38424          0   

   Number of Comments Text    Created On   
0                 651       1.701128e+09  \
1                 651       1.701128e+09   
2                 651       1.701128e+09   
3                 366       1.700921e+09   
4                 366       1.700921e+09   
5                 366       1.70092

In [ ]:
# not necessary
'''
# Change to Make for Upvotes and Downvotes
estimated_upvotes = round(post.score / (2 * post.upvote_ratio - 1)) if post.upvote_ratio != 0.5 else post.score
estimated_downvotes = estimated_upvotes - post.score
'''